In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline
import math

In [2]:
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
import keras
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold

Using Theano backend.


In [3]:
path = '/axp/rim/imsadsml/warehouse/sagra39/Kaggle/ship_vs_iceberg/'
train = pd.read_json(path + 'train.json')
test = pd.read_json(path + 'test.json')

In [4]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [5]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [6]:
Y = train['is_iceberg']
X.shape

(1604, 75, 75, 3)

In [7]:
X = X.astype('float32')
X_test = X_test.astype('float32')

## Train data Split

In [8]:
from sklearn.model_selection import train_test_split
x_train , x_valid , y_train  , y_valid  = train_test_split(X, Y, test_size=0.20, random_state=42)
print(x_train.shape[0], 'train samples')
print(x_valid.shape[0], 'test samples')

1283 train samples
321 test samples


## CNN Model

In [9]:
img_rows, img_cols = 75, 75
input_shape = (img_rows, img_cols, 3)

model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = input_shape))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation = "sigmoid"))


# model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
# model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
# model.add(Dropout(0.2))

# model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(Dropout(0.2))

# model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(Dropout(0.3))

# model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(Dropout(0.3))

# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.2))

# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.2))

# model.add(Dense(1, activation="sigmoid"))

In [11]:
# model.summary()

In [10]:
Adam = keras.optimizers.Adam(lr=0.005)
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=Adam,
              metrics=['accuracy'])

## Data Augmentation

In [65]:
# datagen = ImageDataGenerator(
#         featurewise_center=False,  # set input mean to 0 over the dataset
#         samplewise_center=False,  # set each sample mean to 0
#         featurewise_std_normalization=False,  # divide inputs by std of the dataset
#         samplewise_std_normalization=False,  # divide each input by its std
#         zca_whitening=False,  # apply ZCA whitening
#         rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
#         zoom_range = 0.20, # Randomly zoom image 
#         width_shift_range=0.05,  # randomly shift images horizontally (fraction of total width)
#         height_shift_range=0.05,  # randomly shift images vertically (fraction of total height)
#         horizontal_flip=False,  # randomly flip images
#         vertical_flip=False)  # randomly flip images

# datagen.fit(x_train)

In [11]:
import time
start_time = time.time()
model.fit(x_train, y_train, batch_size=50, epochs=10, verbose=2, validation_data=(x_valid, y_valid))
print (time.time() - start_time)

Train on 1283 samples, validate on 321 samples
Epoch 1/10
25s - loss: 8.4702 - acc: 0.4614 - val_loss: 7.7974 - val_acc: 0.5109
Epoch 2/10
24s - loss: 8.6236 - acc: 0.4591 - val_loss: 7.7974 - val_acc: 0.5109
Epoch 3/10
24s - loss: 8.6236 - acc: 0.4591 - val_loss: 7.7974 - val_acc: 0.5109
Epoch 4/10
24s - loss: 8.6361 - acc: 0.4583 - val_loss: 7.7974 - val_acc: 0.5109
Epoch 5/10
24s - loss: 8.6236 - acc: 0.4591 - val_loss: 7.7974 - val_acc: 0.5109
Epoch 6/10
24s - loss: 8.6111 - acc: 0.4599 - val_loss: 7.7974 - val_acc: 0.5109
Epoch 7/10
24s - loss: 8.6111 - acc: 0.4599 - val_loss: 7.7974 - val_acc: 0.5109
Epoch 8/10
25s - loss: 8.6237 - acc: 0.4591 - val_loss: 7.7974 - val_acc: 0.5109
Epoch 9/10
24s - loss: 8.6236 - acc: 0.4591 - val_loss: 7.7974 - val_acc: 0.5109
Epoch 10/10
24s - loss: 8.6236 - acc: 0.4591 - val_loss: 7.7974 - val_acc: 0.5109
258.8125557899475


In [80]:
# import time
# start_time = time.time()
# history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=100), epochs = 2, validation_data = (x_valid,y_valid), steps_per_epoch=x_train.shape[0] // 50, verbose = 2)
# print (time.time() - start_time)

## Evaluate Model

In [81]:
score = model.evaluate(x_valid, y_valid, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 7.79736571149
Test accuracy: 0.510903426791


## Predict Test

In [69]:
predicted_test=model.predict_proba(X_test, verbose=0)
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv(path + 'sub7.csv', index=False)

## Keras Kfold Cross Validation

In [84]:
model = Sequential()
def getModel():
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = input_shape))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same', activation ='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(256, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation = "sigmoid"))
    
    return model

In [85]:
kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=7)
cvscores = []
for train, test in kfold.split(X, Y):
    
    model=getModel()
    Adam = keras.optimizers.Adam(lr=0.0005)
    model.compile(loss=keras.losses.binary_crossentropy, optimizer=Adam, metrics=['accuracy'])
    
    start_time = time.time()
    model.fit(x_train, y_train, batch_size=500, epochs=1, verbose=10, validation_data=(x_valid, y_valid))
    print (time.time() - start_time)
    
    score = model.evaluate(x_valid, y_valid, verbose=0)
    print('Test loss:', score[0], 'Test accuracy:', score[1])
    print (score)
    cvscores.append(score)
    


Train on 1283 samples, validate on 321 samples
Epoch 1/1
30.23143768310547
Test loss: 0.729509150128 Test accuracy: 0.489096573209
[0.72950915012775552, 0.48909657320872274]


ValueError: Input 0 is incompatible with layer conv2d_23: expected ndim=4, found ndim=2

In [ ]:
print("Mean Scores" (numpy.mean(cvscores)))

## Ensemble

In [16]:
path_new = '/axp/rim/imsadsml/warehouse/sagra39/Kaggle/ship_vs_iceberg/'
import pandas as pd
import numpy as np

file1 = pd.read_csv(path_new + 'baseline.csv')
file2 = pd.read_csv(path_new + 'model_1.csv')
file3 = pd.read_csv(path_new + 'sub01445.csv')
file4 = pd.read_csv(path_new + 'vgg11.csv')
file5 = pd.read_csv(path_new + 'keras+TF.csv')
file6 = pd.read_csv(path_new + 'resnet18.csv')
file7 = pd.read_csv(path_new + 'resnet34.csv')


file1.columns = ['id','target_z']
file2.columns = ['id','target_y']
file3.columns = ['id','target_x']
file4.columns = ['id','target_w']
file5.columns = ['id','target_v']
file6.columns = ['id','target_u']
file7.columns = ['id','target_t']

In [17]:
concat_sub = file1.merge(file2, on='id', how='inner').merge(file3, on='id', how='inner').merge(file4, on='id', how='inner').merge(file5, on='id', how='inner').merge(file6, on='id', how='inner').merge(file7, on='id', how='inner')

In [18]:
concat_sub.head(2)

,id,target_z,target_y,target_x,target_w,target_v,target_u,target_t
0,5941774d,0.067129,0.057523,0.251230,0.058787,0.005586,0.046429,0.068197
1,4023181e,0.420880,0.164504,0.795719,0.582027,0.145927,0.658328,0.810365


In [19]:
models = 8
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:models].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:models].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:models].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:models].median(axis=1)

In [20]:
cutoff_lo = 0.6
cutoff_hi = 0.4
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:models] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:models] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))

In [21]:
concat_sub[['id', 'is_iceberg']].to_csv(path_new + 'baseline_model1_sub01445_vgg11_keras+TF_resnet18_resnet34_stackMinmaxBestbase_06_04.csv', index=False)

In [16]:
import math
Ensemble_cnn_sub8 = CNN.merge(sub8, on='id', how='inner')
# Ensemble_cnn_sub8['is_iceberg'] = Ensemble_cnn_sub8.apply(lambda row: 1/(1+math.exp(-(math.log(row['target_z']/(1-row['target_z']))+math.log(row['target_y']/(1-row['target_y'])))/2)), axis=1)

In [17]:
Ensemble_cnn_sub8['is_iceberg'] = Ensemble_cnn_sub8.apply(lambda row: (row['target_z'] + row['target_y'])/2, axis=1)
Ensemble_cnn_sub8[['id', 'is_iceberg']].to_csv(path_new + 'submission_sub4_sub5_avg.csv', index=False)

In [104]:
sub8[sub8.target_y==0].head()

,id,target_y
1414,5bcc2bf5,0.0
4530,6f32bdde,0.0
5425,bffb70e2,0.0


In [106]:
CNN[CNN.target_z==0].head()

,id,target_z
1414,5bcc2bf5,0.0
4530,6f32bdde,0.0
